In [1]:
from pinecone import Pinecone
from dotenv import load_dotenv
import os

load_dotenv()

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [2]:
index_name='wine-reviews'
index = pc.Index(index_name)

index.describe_index_stats()

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wine_reviews_ns1': {'vector_count': 129971}},
 'total_vector_count': 129971,
 'vector_type': 'dense'}

In [3]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader("./winemag-data-130k-v2.csv")
docs = loader.load()

for i, doc in enumerate(docs[:3]):
    print(str(i), doc)

0 page_content=': 0
country: Italy
description: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
designation: Vulkà Bianco
points: 87
price: 
province: Sicily & Sardinia
region_1: Etna
region_2: 
taster_name: Kerin O’Keefe
taster_twitter_handle: @kerinokeefe
title: Nicosia 2013 Vulkà Bianco  (Etna)
variety: White Blend
winery: Nicosia' metadata={'source': './winemag-data-130k-v2.csv', 'row': 0}
1 page_content=': 1
country: Portugal
description: This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.
designation: Avidagos
points: 87
price: 15.0
province: Douro
region_1: 
region_2: 
taster_name: Roger Voss
taster_twitter_handle: @vossroger
title: Quinta dos Avidagos 2011 Avidagos Red (Douro)
varie

In [5]:
vars(docs[0])

{'id': None,
 'metadata': {'source': './winemag-data-130k-v2.csv', 'row': 0},
 'page_content': ": 0\ncountry: Italy\ndescription: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.\ndesignation: Vulkà Bianco\npoints: 87\nprice: \nprovince: Sicily & Sardinia\nregion_1: Etna\nregion_2: \ntaster_name: Kerin O’Keefe\ntaster_twitter_handle: @kerinokeefe\ntitle: Nicosia 2013 Vulkà Bianco  (Etna)\nvariety: White Blend\nwinery: Nicosia",
 'type': 'Document'}

## 벡터스토어를 이용해 임베딩 & 적재

In [7]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

In [7]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_documents(
    docs, 
    embedding, 
    index_name="wine-reviews", 
    namespace="wine_reviews_ns1"
)

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


### 기존 인덱스 불러오기

In [8]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_existing_index(
    index_name="wine-reviews",     # 어제 만든 인덱스 이름
    embedding=embedding,
    namespace="wine_reviews_ns1"   # 어제 지정한 네임스페이스
)

In [9]:
results = vector_store.similarity_search(
    "달콤한 맛을 느낄 수 있는 와인", 
    k=5,
    namespace="wine_reviews_ns1"
)

results

[Document(id='e41efd07-36ce-40d5-9d91-365bcd6b5eef', metadata={'row': 47937.0, 'source': './winemag-data-130k-v2.csv'}, page_content=': 47937\ncountry: US\ndescription: As sweet as honey, with Lifesaver candy pineapple, golden apricot preserves, Meyer lemon custard pie and Asian spice flavors. Calls for very rich fare, such as scallops sautéed in butter, served with a creamy risotto.\ndesignation: \npoints: 88\nprice: 22.0\nprovince: California\nregion_1: Sonoma County\nregion_2: Sonoma\ntaster_name: \ntaster_twitter_handle: \ntitle: Wellington 2007 Roussanne (Sonoma County)\nvariety: Roussanne\nwinery: Wellington'),
 Document(id='9406e6ec-b63f-4d20-b831-10bb8fbd8656', metadata={'row': 45019.0, 'source': './winemag-data-130k-v2.csv'}, page_content=": 45019\ncountry: US\ndescription: Once again this designate amazes with a surfeit of beautifully integrated flavors. Mushrooms and baking spices adorn an explosion of strawberry pastries. There's plenty of ripe cherry fruit also, and a long

## Search_wine

In [22]:
import os

def search_wine(dish_flavor):
    results = vector_store.similarity_search(
        dish_flavor,
        k=5,
        namespace=os.getenv("PINECONE_NAMESPACE")
    )

    return {
        "dish_flavor": dish_flavor,
        "wine_reviews": "\n".join([doc.page_content for doc in results])
    }


In [23]:
from langchain.schema.runnable import RunnableLambda


taste_query =  "이 요리는 판차넬라 샐러드로, 신선한 토마토와 바질의 상큼함이 빵의 고소함과 어우러져 상쾌하고 풍부한 맛을 냅니다."
# search_wine 함수는 이전 셀에서 정의한 함수 사용
runnable = RunnableLambda(search_wine)

# taste_query는 검색할 쿼리(예: "달콤한 치킨 요리")
response = runnable.invoke(taste_query)

# 결과 출력
print(response["dish_flavor"])
print(response["wine_reviews"])


이 요리는 판차넬라 샐러드로, 신선한 토마토와 바질의 상큼함이 빵의 고소함과 어우러져 상쾌하고 풍부한 맛을 냅니다.



## Recommend_wine

In [25]:
def recommend_wine_chain(query):
    
  chat_template = ChatPromptTemplate.from_messages(
    [
      ('system', SYSTEM_PROMPT),
      ('human', [ {'type': 'text', 'text':query['text']},
                  {'type':'image_url', 'image_url':  {'url': query['image_url']}}])
    ]
  )
  chain = chat_template|llm|output_parser
  return chain

## chain 연결

In [26]:
runnable_1=RunnableLambda(describe_dish_flavor_chain)
runnable_2=RunnableLambda(search_wine)
runnable_3=RunnableLambda(recommend_wine_chain)

chain=runnable_1|runnable_2|runnable_3

NameError: name 'describe_dish_flavor_chain' is not defined

In [ ]:
chain.get_graph().print_ascii()

In [ ]:
%pip install -qU grandalf

In [ ]:
#음식이미지 링크> 음식찾아서 음식맛> 음식맛이 와인평에 있는 와인검색>와인평+음식맛>소믈리에메세지
chain.get_graph().print_ascii()

In [ ]:
response=chain.invoke(
    {
        "image_url":"https://www.stockfood.com/Sites/StockFood/Documents/Homepage/News//en/16.jpg"
    }
    
    
)

In [ ]:
response = chain.invoke(query_2)

response